# Categorical predictors

In this notebook we work with categorical predictors to define a regression model.

Let's import the packages and load the auto-mpg dataset into a pandas dataframe

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

df = pd.read_csv('./data/auto-mpg.csv')
df.head()


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


The auto-mpg dataset has two categorical predictors: **origin** and the **brand** of the car which can be extracted from the **name** variable.


In [19]:
df.origin.value_counts()

1    245
3     79
2     68
Name: origin, dtype: int64

In [4]:
df['brand'] = df.name.apply(lambda d : d.split(' ')[0])
df.brand.value_counts()


ford             48
chevrolet        43
plymouth         31
dodge            28
amc              27
toyota           25
datsun           23
buick            17
pontiac          16
volkswagen       15
honda            13
mercury          11
mazda            10
oldsmobile       10
peugeot           8
fiat              8
audi              7
chrysler          6
vw                6
volvo             6
saab              4
opel              4
subaru            4
renault           3
chevy             3
bmw               2
cadillac          2
maxda             2
mercedes-benz     2
mercedes          1
nissan            1
triumph           1
capri             1
vokswagen         1
toyouta           1
chevroelt         1
hi                1
Name: brand, dtype: int64

Let's see how the origin variable can be used to model the mpg outcome variable

But, we first need to convert the origin variable which is an integer
into a string type category to avoid introducing
an artificial order on the origin of the car.


In [5]:
origin = { 1: 'American', 2: 'European', 3: 'Japanese'}
df['origin'] = df.origin.apply(lambda d : origin[d])
df.origin.value_counts()

df.head()


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
0,18.0,8,307.0,130.0,3504.0,12.0,70,American,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165.0,3693.0,11.5,70,American,buick skylark 320,buick
2,18.0,8,318.0,150.0,3436.0,11.0,70,American,plymouth satellite,plymouth
3,16.0,8,304.0,150.0,3433.0,12.0,70,American,amc rebel sst,amc
4,17.0,8,302.0,140.0,3449.0,10.5,70,American,ford torino,ford


## Dummy Encoding
In general, to handle categorical variables we need to **dummy encode** the categories.
Meaning that for N categories we need to create N-1 binary variables.
We can do that with the ```pandas get_dummies``` function



In [6]:
pd.get_dummies(df.origin).head()

,American,European,Japanese
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


This creates 3 new variables indicating whether the car is or is not American, European or Japanese.

These variables can be merged into the original dataframe


In [7]:
df = df.merge(pd.get_dummies(df.origin), left_index = True, right_index = True)
df.head()


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand,American,European,Japanese
0,18.0,8,307.0,130.0,3504.0,12.0,70,American,chevrolet chevelle malibu,chevrolet,1,0,0
1,15.0,8,350.0,165.0,3693.0,11.5,70,American,buick skylark 320,buick,1,0,0
2,18.0,8,318.0,150.0,3436.0,11.0,70,American,plymouth satellite,plymouth,1,0,0
3,16.0,8,304.0,150.0,3433.0,12.0,70,American,amc rebel sst,amc,1,0,0
4,17.0,8,302.0,140.0,3449.0,10.5,70,American,ford torino,ford,1,0,0


Now we build a simple linear regression model

In [8]:
result = smf.ols('mpg ~ Japanese + European', data = df).fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     96.60
Date:                Thu, 18 Jul 2019   Prob (F-statistic):           8.67e-35
Time:                        16:27:53   Log-Likelihood:                -1282.2
No. Observations:                 392   AIC:                             2570.
Df Residuals:                     389   BIC:                             2582.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     20.0335      0.409     49.025      0.000      19.230      20.837
Japanese      10.4172      0.828     12.588      0.000       8.790      12.044
European       7.5695      0.877      8.634      0.000       5.846       9.293
==============================================================================
Omnibus:                       26.330   Durbin-Watson:                   0.763
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.217
Skew:                           0.679   Prob(JB):                     2.74e-07
Kurtosis:                       3.066   Cond. No.                         3.16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In fact when using the statsmodels library, we don't need to manually dummy encode the categorical variables.

We can simply define the model as

```mpg ~ C(origin)```



In [9]:
result = smf.ols('mpg ~ C(origin) ', data = df).fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     96.60
Date:                Thu, 18 Jul 2019   Prob (F-statistic):           8.67e-35
Time:                        16:28:26   Log-Likelihood:                -1282.2
No. Observations:                 392   AIC:                             2570.
Df Residuals:                     389   BIC:                             2582.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                20.0335      0.409     49.025      0.000      19.230      20.837
C(origin)[T.European]     7.5695      0.877      8.634      0.000       5.846       9.293
C(origin)[T.Japanese]    10.4172      0.828     12.588      0.000       8.790      12.044
==============================================================================
Omnibus:                       26.330   Durbin-Watson:                   0.763
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.217
Skew:                           0.679   Prob(JB):                     2.74e-07
Kurtosis:                       3.066   Cond. No.                         3.16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

And we get the exact same model.

The C function that we just used, tells the library
that the variable is categorical
and forces the creation of the dummy variables.

Let's take a closer look at the values of the coefficients for that model




In [10]:
result.params

Intercept                20.033469
C(origin)[T.European]     7.569472
C(origin)[T.Japanese]    10.417164
dtype: float64

and compare these coefficients with the average mpg per origin:


In [11]:
df[['mpg','origin']].groupby(by = 'origin').mean()


,mpg
origin,
American,20.033469
European,27.602941
Japanese,30.450633


It's easy to see that:
the intercept coefficient equals the mean of the left out category (American cars)

and the other coefficients when added to the intercept equal the average of the target variable for that category.


In [14]:
print("European coefficient + Intercept:")
print(result.params['C(origin)[T.European]'] + result.params['Intercept'] )


print("Average of mpg for  European cars")
print(df[['mpg','origin']].groupby(by = 'origin').mean().reset_index().loc[1]['mpg'])


European coefficient + Intercept:
27.602941176470605
Average of mpg for  European cars
27.602941176470587


Let's now work on the brand categorical variable


In [15]:
df.brand.value_counts().shape

(37,)

We have 37 different brands. Way more than the three origin categories.
In that case the dummy encoding technique will not give good results since each brand won't have enough samples.

We can use another encoding technique called binary encoding.

The idea is to order the categories and then encode the order number into a binary number.

Since 2 to the power of 5 equals 32 and we have 37 different brands we need 6 digits to binary encode the category order.


In [20]:
import category_encoders as ce
# reload the dataset
df = pd.read_csv('./data/auto-mpg.csv')
df['brand'] = df.name.apply(lambda d : d.split(' ')[0])

# define the encoder
encoder = ce.BinaryEncoder(cols=['brand'])
df = encoder.fit_transform(df)

df.head()

,brand_0,brand_1,brand_2,brand_3,brand_4,brand_5,brand_6,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,0,0,0,0,0,0,1,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,0,0,0,0,0,1,0,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,0,0,0,0,0,1,1,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,0,0,0,0,1,0,0,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,0,0,0,0,1,0,1,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


This adds 7 new variables brand_0, brand_1 up to brand_6 to the dataframe.

In [21]:
df.columns

Index(['brand_0', 'brand_1', 'brand_2', 'brand_3', 'brand_4', 'brand_5',
       'brand_6', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name'],
      dtype='object')

We can now define the model

```mpg ~ brand_0 + brand_1 + ... up to brand_6```


In [22]:
result = smf.ols('mpg ~ brand_0 + brand_1 + brand_2 + brand_3 + brand_4 + brand_5 + brand_6 ', data = df).fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     26.37
Date:                Thu, 18 Jul 2019   Prob (F-statistic):           2.69e-26
Time:                        16:33:26   Log-Likelihood:                -1293.7
No. Observations:                 392   AIC:                             2601.
Df Residuals:                     385   BIC:                             2629.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0814      0.882     21.635      0.000      17.347      20.815
brand_0     -2.87e-17   2.19e-16     -0.131      0.896   -4.59e-16    4.02e-16
brand_1        6.7307      2.402      2.802      0.005       2.008      11.453
brand_2        3.1976      0.820      3.901      0.000       1.586       4.809
brand_3        8.1270      0.791     10.278      0.000       6.572       9.682
brand_4       -0.4654      0.721     -0.646      0.519      -1.883       0.952
brand_5        1.9886      0.687      2.895      0.004       0.638       3.339
brand_6        0.6642      0.712      0.933      0.351      -0.735       2.063
==============================================================================
Omnibus:                       11.111   Durbin-Watson:                   0.695
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.550
Skew:                           0.420   Prob(JB):                      0.00310
Kurtosis:                       2.991   Cond. No.                     2.00e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.99e-34. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

and we get a r-squared of 0.28

However, we lost any possibility to interpret the coefficients.
